In [2]:
from skt.gcp import (
    PROJECT_ID,
    bq_insert_overwrite,
    bq_to_df,
    bq_to_pandas,
    get_bigquery_client,
    bq_table_exists,
    get_max_part,
    load_query_result_to_table,
    pandas_to_bq,
    pandas_to_bq_table,
    load_bigquery_ipython_magic,
    get_bigquery_client,
    _print_query_job_results,
    load_query_result_to_partitions
    
)

from skt.ye import (
    get_hdfs_conn,
    get_spark,
    hive_execute,
    hive_to_pandas,
    pandas_to_parquet,
    slack_send,
    get_secrets
)

In [3]:
from datetime import date, datetime, timedelta
import time
from datetime import date, datetime

import numpy as np
import pandas as pd
import pyarrow.parquet as pq
from dateutil.relativedelta import relativedelta
from pyhive import hive

from copy import deepcopy
from joblib import Parallel, delayed
import warnings

from tqdm.notebook import tqdm

DT :  2024-06-11
DT_threshold :  2024-04-12
DT_threshold :  2024-06-04


In [ ]:
print(f'current_dt: {current_dt}')
print(f'state: {state}')

In [ ]:
execution_dt = datetime.strptime(current_dt, '%Y-%m-%d')
execution_dt_one_ago = (execution_dt - timedelta(days=1))
lag_current_dt= execution_dt_one_ago.strftime('%Y-%m-%d')
print(f'execution_dt: {lag_current_dt}')

In [ ]:
PROJECT_ID = "skt-datahub"
data_db_name = "adot_reco"
save_db_name = "adot_reco_dev"
table_nm = "recgpt_log_sequence_daily_preprocessed_prd"
lag_table_nm = "recgpt_log_sequence_lag_daily_preprocessed_prd"

In [ ]:
query = f"""
WITH TDEAL_TABLE AS (
    SELECT  svc_mgmt_num,
            luna_id,
            "tdeal" as source,
            TRIM(LOWER(cat2)) as cat2,
            TRIM(LOWER(cat3)) as cat3,
            unix_time,
            CASE 
                WHEN EXTRACT(DAYOFWEEK FROM dt) IN (1, 7) THEN 1
                ELSE 0
            END AS is_weekend,
            dt

    FROM {data_db_name}.{lag_table_nm}
    WHERE dt = '{lag_current_dt}' and type = 'tdeal_v3'
)

SELECT  svc_mgmt_num,
        luna_id,
        cat2,
        cat3,
        count(*) as cat2_cnt,
        is_weekend,
        dt
FROM TDEAL_TABLE
group by svc_mgmt_num, luna_id, cat2, cat3, dt, is_weekend
"""

In [ ]:
bq_insert_overwrite(sql=query, destination=f'{PROJECT_ID}.{save_db_name}.tdeal_cat2_cnt', partition='dt')